# Imports

In [1]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
)

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Layer, RepeatVector
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences



# 3-Tank-Prozess

In [2]:
res = pd.read_csv("3Tank_Experiment_Batch.csv")

In [3]:
res

,t,Sequenz,u1(t),u2(t),x1(t),x2(t),x3(t),v(t)
0,0,1,0.000000e+00,0.000000,5.988230,2.331288,1.212181,4.544362
1,1,1,2.194408e+00,2.394381,1.467460,1.213719,1.478458,4.936056
2,2,1,4.196763e+00,4.603298,1.314415,1.622736,1.535122,5.327472
3,3,1,6.017419e+00,6.635741,1.491854,1.677920,1.598870,5.335756
4,4,1,7.666488e+00,8.500489,1.719355,1.564766,1.889846,5.568579
...,...,...,...,...,...,...,...,...
27995,135,200,2.500000e+01,3.015062,7.120053,6.206604,0.558532,3.198816
27996,136,200,2.500000e+01,2.565913,6.950856,6.501144,0.482466,3.168365
27997,137,200,1.775956e+01,2.096692,7.062912,6.718728,0.375799,2.985077
27998,138,200,7.673862e-13,1.606469,7.550670,6.683511,0.640433,2.220530


A1 = A2 = A3, Grundfläche der Tanks<br>
q1, q2, q3 : Querschnitt der Durchflussrohre<br>
q1 = 2, q2 = 2, q3 = 2<br>

In [4]:
px.line(res, x="t", y="v(t)", color="Sequenz", title="Füllstand v(t) über die Zeit")

<br>

# Vorhersage der gesamten Abflussmenge aus Tank 3 am Ende jedes Batches

## Feature- und Target-Generierung

In [5]:
# # Annahme: Zeitabstand zwischen den Messungen in t ist konstant und beträgt delta_t (z.B. 1 Sekunde)
# delta_t = 1  

# def abflussmenge_pro_batch(grp):
#     # Integral (Summe) von v(t) * delta_t für jede Sequenz
#     return (grp["v(t)"]*delta_t).sum()

# # Neue Zielvariable erzeugen: gesamte Abflussmenge pro Batch (jedem Zeitschritt der Batch-Sequenz zuordnen)
# abfluss_dict = res.groupby("Sequenz").apply(abflussmenge_pro_batch).to_dict()
# res["V_total_out"] = res["Sequenz"].map(abfluss_dict)

In [6]:
delta_t = 1
durchmesser = 2            # in LE
radius = durchmesser / 2   # in LE
A = np.pi * radius**2      # in LE^2

def abflussmenge_pro_batch(grp):
    return (grp["v(t)"] * A * delta_t).sum()

abfluss_dict = res.groupby("Sequenz").apply(abflussmenge_pro_batch).to_dict()
res["V_total_out"] = res["Sequenz"].map(abfluss_dict)

C:\Users\sinan\AppData\Local\Temp\ipykernel_22296\1335688016.py:9: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [7]:
res

,t,Sequenz,u1(t),u2(t),x1(t),x2(t),x3(t),v(t),V_total_out
0,0,1,0.000000e+00,0.000000,5.988230,2.331288,1.212181,4.544362,2863.896813
1,1,1,2.194408e+00,2.394381,1.467460,1.213719,1.478458,4.936056,2863.896813
2,2,1,4.196763e+00,4.603298,1.314415,1.622736,1.535122,5.327472,2863.896813
3,3,1,6.017419e+00,6.635741,1.491854,1.677920,1.598870,5.335756,2863.896813
4,4,1,7.666488e+00,8.500489,1.719355,1.564766,1.889846,5.568579,2863.896813
...,...,...,...,...,...,...,...,...,...
27995,135,200,2.500000e+01,3.015062,7.120053,6.206604,0.558532,3.198816,2791.727728
27996,136,200,2.500000e+01,2.565913,6.950856,6.501144,0.482466,3.168365,2791.727728
27997,137,200,1.775956e+01,2.096692,7.062912,6.718728,0.375799,2.985077,2791.727728
27998,138,200,7.673862e-13,1.606469,7.550670,6.683511,0.640433,2.220530,2791.727728


In [8]:
# DATEN-PIPELINE
features = ["u1(t)", "u2(t)", "v(t)", "x1(t)", "x2(t)"]
target = "V_total_out"

print(f"Features: {features}")
print(f"Target: {target}")


# Daten in Sequenzen umwandeln
unique_sequences = res["Sequenz"].unique()
n_sequences = len(unique_sequences)
n_timesteps = res.groupby("Sequenz").size().iloc[0]
n_features = len(features)

# Initialisiere die leeren NumPy-Arrays
X = np.zeros((n_sequences, n_timesteps, n_features))
y = np.zeros((n_sequences, 1))

print(f"\nErstelle 3D-Tensoren mit der Form (Batches, Zeitschritte, Features)...")
for i, seq_id in enumerate(unique_sequences):
    # Extrahiere die Features für die aktuelle Sequenz
    sequence_data = res[res["Sequenz"] == seq_id][features].values
    X[i] = sequence_data

    # Extrahiere den Zielwert (ist für die ganze Sequenz gleich, also nehmen wir den ersten)
    target_value = res[res["Sequenz"] == seq_id][target].iloc[0]
    y[i] = target_value

print(f"Form des Feature-Tensors X: {X.shape}")
print(f"Form des Target-Vektors y: {y.shape}")

Features: ['u1(t)', 'u2(t)', 'v(t)', 'x1(t)', 'x2(t)']
Target: V_total_out

Erstelle 3D-Tensoren mit der Form (Batches, Zeitschritte, Features)...
Form des Feature-Tensors X: (200, 140, 5)
Form des Target-Vektors y: (200, 1)


## Aufteilen in Trainings-, Validierungs- und Testdaten (Batch-weise)

In [9]:
# Wir teilen die Indizes der Batches auf, nicht die einzelnen Zeilen!
indices = np.arange(n_sequences)

# Zuerst 80% Training und 20% für (Validierung + Test)
X_train, X_temp, y_train, y_temp, indices_train, indices_temp = train_test_split(
    X, y, indices, test_size=0.2, random_state=42
)

# Dann die 20% in 10% Validierung und 10% Test aufteilen (50/50 split von temp)
X_val, X_test, y_val, y_test, indices_val, indices_test = train_test_split(
    X_temp, y_temp, indices_temp, test_size=0.5, random_state=42
)

print("\nDaten aufgeteilt:")
print(f"Trainingsdaten: {X_train.shape[0]} Batches")
print(f"Validierungsdaten: {X_val.shape[0]} Batches")
print(f"Testdaten: {X_test.shape[0]} Batches")


Daten aufgeteilt:
Trainingsdaten: 160 Batches
Validierungsdaten: 20 Batches
Testdaten: 20 Batches


## Skalierung der Daten

In [10]:
# Skalierung für die Features (X)
scaler_X = StandardScaler()
X_train_reshaped = X_train.reshape(-1, n_features)
scaler_X.fit(X_train_reshaped)

# Transformation anwenden und zurück in 3D-Form bringen
X_train_scaled = scaler_X.transform(X_train_reshaped).reshape(X_train.shape)
X_val_scaled = scaler_X.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, n_features)).reshape(X_test.shape)

# Skalierung für den Zielwert (y)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)
y_test_scaled = scaler_y.transform(y_test)


print(f"X_train_scaled: {X_train_scaled.shape}, y_train_scaled: {y_train_scaled.shape}")
print(f"X_val_scaled: {X_val_scaled.shape}, y_val_scaled: {y_val_scaled.shape}")
print(f"X_test_scaled: {X_test_scaled.shape}, y_test_scaled: {y_test_scaled.shape}")

X_train_scaled: (160, 140, 5), y_train_scaled: (160, 1)
X_val_scaled: (20, 140, 5), y_val_scaled: (20, 1)
X_test_scaled: (20, 140, 5), y_test_scaled: (20, 1)


<br>

## Modellerstellung und -training

### STA-LSTM

- Das Modell erkennt automatisch:
    - **Welche Sensoren** zu welchem **Zeitpunkt** für die Qualitätsvorhersage wichtig sind.
    - **Welche Zeitpunkte** im bisherigen Prozessverlauf besonders stark auf die Zielgröße wirken.

In [11]:
# --- Custom Layer: Spatial Attention ---
class SpatialAttention(Layer):
    """
    Berechnet pro Zeitschritt eine Gewichtung für jede Eingabevariable (Feature).
    Die wichtigsten Variablen pro Zeitschritt werden vom Modell gelernt.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense = Dense(units=1)

    def call(self, x):
        # x: (batch, time, features)
        attn_scores = tf.nn.softmax(self.dense(x), axis=2)  # Gewichtung über Features
        return x * attn_scores  # Elementweise Multiplikation: Features gewichten

# --- Custom Layer: Temporal Attention ---
class TemporalAttention(Layer):
    """
    Berechnet für die LSTM-Hidden-States eine Gewichtung über alle Zeitschritte.
    So kann das Modell entscheiden, welche Zeitpunkte für die Qualitätsprognose am wichtigsten sind.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dense = Dense(units=1)

    def call(self, h_seq):
        # h_seq: (batch, time, hidden_dim)
        attn_scores = tf.nn.softmax(self.dense(h_seq), axis=1)  # Gewichtung über Zeit
        context = tf.reduce_sum(h_seq * attn_scores, axis=1)  # Kontextvektor: gewichtete Summe über Zeit
        return context

# --- STA-LSTM-Modell-Funktion ---
def build_sta_lstm_model(input_shape, lstm_units=64, dropout_rate=0.3):
    """
    Baut und kompiliert ein STA-LSTM-Modell:
    - Spatial Attention (über Features)
    - LSTM Encoder (über Zeit)
    - Temporal Attention (über Zeit)
    - Decoder: Dense-Layer für die Vorhersage
    """
    inputs = Input(shape=input_shape)
    # 1. Spatial Attention: Welche Variablen pro Zeitschritt sind wichtig?
    x = SpatialAttention()(inputs)
    # 2. LSTM Encoder: Zeitliche Verarbeitung
    h_seq = LSTM(lstm_units, return_sequences=True)(x)
    # 3. Temporal Attention: Welche Zeitpunkte sind wichtig?
    context = TemporalAttention()(h_seq)
    # 4. Optional: Dropout für Robustheit
    dropout_out = Dropout(dropout_rate)(context)
    # 5. Decoder: Dense-Layer, Ausgabe ist die Qualitätsvariable
    outputs = Dense(1)(dropout_out)
    # Modell zusammenbauen
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

# --- Beispiel-Nutzung (wie im Notebook) ---
# input_shape = (seq_len, n_features), z. B. (140, 5)
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
model = build_sta_lstm_model(input_shape)
print("STA-LSTM-Modellarchitektur:")
model.summary()



# Callbacks für ein besseres Training
# 1. EarlyStopping: Beendet das Training, wenn die Validierungs-Loss sich nicht mehr verbessert.
early_stopping = EarlyStopping(
    monitor="val_loss",  # Überwache die Loss auf den Validierungsdaten
    patience=10,  # Anzahl der Epochen ohne Verbesserung, bevor gestoppt wird
    verbose=1,
    restore_best_weights=True,
)  # Stellt die besten Gewichte am Ende wieder her

# 2. ModelCheckpoint: Speichert das beste Modell während des Trainings.
model_checkpoint = ModelCheckpoint(
    "best_model.keras",  # Dateipfad
    monitor="val_loss",
    save_best_only=True,  # Speichere nur, wenn 'val_loss' sich verbessert
    verbose=1,
)

# Trainiere das Modell
print("\nStarte das Modelltraining...")
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_scaled, y_val_scaled),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1,
)
print("Modelltraining abgeschlossen.")


STA-LSTM-Modellarchitektur:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 140, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_attention               │ (None, 140, 5)         │             6 │
│ (SpatialAttention)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 140, 64)        │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ temporal_attention              │ (None, 64)             │            65 │
│ (TemporalAttention)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,056 (70.53 KB)

 Trainable params: 18,056 (70.53 KB)

 Non-trainable params: 0 (0.00 B)


Starte das Modelltraining...
Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.5596 - mean_absolute_error: 0.5996
Epoch 1: val_loss improved from inf to 0.47253, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step - loss: 0.5583 - mean_absolute_error: 0.5986 - val_loss: 0.4725 - val_mean_absolute_error: 0.4940
Epoch 2/50
3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.4515 - mean_absolute_error: 0.5427
Epoch 2: val_loss improved from 0.47253 to 0.20232, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.3778 - mean_absolute_error: 0.4826 - val_loss: 0.2023 - val_mean_absolute_error: 0.3684
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1573 - mean_absolute_error: 0.2925
Epoch 3: val_loss improved from 0.20232 to 0.11948, saving model to best_model.keras
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.1515 - mean_absolute_error: 0.2876 - val_loss: 0.1195 - val_mean_absolute_error: 0.2845
Epoch 4/50
5/5 ━━━━━━━━━━━━━

<br>

## Visualisierung Modellverbesserung über Trainingsverlauf

In [12]:
# Erstelle einen DataFrame aus der Trainingshistorie
history_df = pd.DataFrame(history.history)
history_df["epoch"] = history_df.index + 1

# Plotte die Loss (Mean Squared Error)
fig_loss = px.line(
    history_df,
    x="epoch",
    y=["loss", "val_loss"],
    title="Trainings- & Validierungs-Loss (MSE)",
    labels={"value": "Loss (MSE)", "variable": "Datensatz"},
)
fig_loss.show()

# Plotte die Metrik (Mean Absolute Error)
fig_mae = px.line(
    history_df,
    x="epoch",
    y=["mean_absolute_error", "val_mean_absolute_error"],
    title="Trainings- & Validierungs-MAE",
    labels={"value": "Mean Absolute Error", "variable": "Datensatz"},
)
fig_mae.show()

<br>

## Wahl des besten Modells + Evaluierung

In [13]:
# EVALUIERUNG


# 1. Lade das beste Modell, das während des Trainings gespeichert wurde
print("Lade das beste gespeicherte Modell 'best_model.keras'...")
#best_model = load_model("best_model.keras")
best_model = load_model(
    "best_model.keras",
    custom_objects={
        "SpatialAttention": SpatialAttention,
        "TemporalAttention": TemporalAttention
    }
)

print("Modell geladen.")

# 2. Definiere die Schritte für die Evaluierung (z.B. in 10%-Schritten)
evaluation_points = np.linspace(0.1, 1.0, 10)  # 10%, 20%, ..., 100%
timesteps = X_test_scaled.shape[1]  # 140
mae_per_step = []

print(
    "\nStarte die Evaluierung der Vorhersagegenauigkeit in Abhängigkeit vom Batch-Fortschritt..."
)

for progress in evaluation_points:
    # Berechne, wie viele Zeitschritte dem aktuellen Fortschritt entsprechen
    current_timesteps = int(timesteps * progress)

    # Erstelle eine temporäre, verkürzte Version des Testsets
    X_test_partial = X_test_scaled[:, :current_timesteps, :]

    # Padde die Sequenzen von vorne mit Nullen, damit sie die volle Länge haben
    # Das ist wichtig, da das LSTM eine feste Input-Länge erwartet.
    # 'pre'-padding ist üblich für Zeitreihen.
    X_test_padded = pad_sequences(
        X_test_partial,
        maxlen=timesteps,
        dtype="float32",
        padding="pre",
        truncating="pre",
    )

    # Mache Vorhersagen mit dem Modell auf den (skalierten) gepaddeten Daten
    y_pred_scaled = best_model.predict(X_test_padded)

    # Rück-Transformation der Vorhersagen und der wahren Werte in die Originalskala
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(
        y_test_scaled
    )  # y_test_scaled sind die Targets für X_test_scaled

    # Berechne den Mean Absolute Error für diesen Fortschrittsschritt
    mae = mean_absolute_percentage_error(y_true, y_pred)
    mae_per_step.append(mae)

    print(
        f"Fortschritt: {progress*100:.0f}% ({current_timesteps} Zeitschritte) -> MAE: {mae:.4f}"
    )

print("\nEvaluierung abgeschlossen.")


# 3. Visualisierung des Ergebnisses
eval_df = pd.DataFrame(
    {
        "Batch-Fortschritt (%)": evaluation_points * 100,
        "Mean Absolute Percent Error (MAPE)": mae_per_step,
    }
)

fig_eval = px.line(
    eval_df,
    x="Batch-Fortschritt (%)",
    y="Mean Absolute Percent Error (MAPE)",
    title="Soft-Sensor-Genauigkeit während des Batches",
    markers=True,
)

fig_eval.update_layout(
    yaxis_title="Vorhersagefehler (MAPE) auf dem Testset",
    xaxis_title="Verfügbare Daten vom Batch (%)",
)
fig_eval.show()

Lade das beste gespeicherte Modell 'best_model.keras'...
Modell geladen.

Starte die Evaluierung der Vorhersagegenauigkeit in Abhängigkeit vom Batch-Fortschritt...


c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning:

`build()` was called on layer 'spatial_attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.

c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning:

`build()` was called on layer 'temporal_attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Fortschritt: 10% (14 Zeitschritte) -> MAE: 0.1583
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Fortschritt: 20% (28 Zeitschritte) -> MAE: 0.1533
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Fortschritt: 30% (42 Zeitschritte) -> MAE: 0.1523
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Fortschritt: 40% (56 Zeitschritte) -> MAE: 0.1463
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Fortschritt: 50% (70 Zeitschritte) -> MAE: 0.1362
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Fortschritt: 60% (84 Zeitschritte) -> MAE: 0.1236
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Fortschritt: 70% (98 Zeitschritte) -> MAE: 0.1034
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Fortschritt: 80% (112 Zeitschritte) -> MAE: 0.0788
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Fortschritt: 90% (126 Zeitschritte) -> MAE: 0.0450
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Fortschritt: 100% (140 Zeitschritte) -> MAE: 0.0056

Evaluierung abgeschlossen.


<br>

## Unsicherheit der Vorhersage während eines Batchverlaufs

In [14]:
batch_idx = 5  # Ein Beispiel-Batch (kannst du auch random wählen)
mean_preds = []
std_preds = []

def mc_dropout_predictions(model, X, n_iter=50):
    preds = []
    for _ in range(n_iter):
        preds.append(model(X, training=True).numpy().squeeze())     #Modellvorhersage mit aktiviertem Dropout, um Unsicherheit zu schätzen (training=True)
    preds = np.array(preds)
    mean = preds.mean(axis=0)
    std = preds.std(axis=0)
    return mean, std

for t_cut in range(10, timesteps + 1, 10):
    X_batch = X_test_scaled[batch_idx:batch_idx+1, :t_cut, :]
    X_padded = pad_sequences(
        X_batch, maxlen=timesteps, dtype="float32", padding="pre", truncating="pre"
    )
    m, s = mc_dropout_predictions(best_model, X_padded, n_iter=50)
    mean_preds.append(scaler_y.inverse_transform(m.reshape(-1, 1)).flatten()[0])
    std_preds.append(s * scaler_y.scale_[0])
timesteps_list = list(range(10, timesteps + 1, 10))

fig4 = go.Figure()
fig4.add_trace(go.Scatter(
    x=timesteps_list,
    y=mean_preds,
    mode='lines+markers',
    name="Vorhersage"
))
fig4.add_trace(go.Scatter(
    x=timesteps_list,
    y=np.array(mean_preds) + np.array(std_preds),
    fill=None,
    mode='lines',
    line_color='lightblue',
    showlegend=False
))
fig4.add_trace(go.Scatter(
    x=timesteps_list,
    y=np.array(mean_preds) - np.array(std_preds),
    fill='tonexty',
    mode='lines',
    line_color='lightblue',
    fillcolor='rgba(135,206,250,0.2)',
    name="Unsicherheitsband (±1 StdAbw.)"
))
fig4.update_layout(
    title="Vorhersage und Unsicherheitsband für einen Beispiel-Batch",
    xaxis_title="Zeitschritt",
    yaxis_title="Vorhergesagte Qualitätsvariable (Originalskala)"
)
fig4.show()


c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:241: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 140, 5))

c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:241: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 140, 5))

c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\functional.py:241: UserWarning:

The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 140, 5))



KeyboardInterrupt: 

<br>

## Simulation eines neuen, unbekannten Batches

1. Wir wählen einen Beispiel-Batch aus unserem Testset (dies simuliert einen "neuen, unbekannten" Batch).
2. Wir "spulen die Zeit vor": Wir nehmen zuerst nur die ersten 10% der Daten dieses Batches, dann die ersten 20%, und so weiter.
3. Wir füttern das Modell mit diesen unvollständigen Daten und lassen es eine Vorhersage für den Endwert treffen.
4. Wir vergleichen die Vorhersage mit dem wahren, aber eigentlich "unbekannten" Endwert dieses Batches.

In [ ]:
# VORHERSAGE WÄHREND EINES BATCHES


# 1. SETUP: Lade Modell und Scaler und wähle einen Beispiel-Batch aus
# Lade das trainierte Modell und die Scaler (stelle sicher, dass die vorherigen Zellen gelaufen sind)
best_model = load_model(
    "best_model.keras",
    custom_objects={
        "SpatialAttention": SpatialAttention,
        "TemporalAttention": TemporalAttention
    }
)# scaler_X und scaler_y sind bereits aus der Pipeline-Zelle im Speicher

# Wähle einen zufälligen Batch aus dem Testset für unsere Demonstration
# Wir nehmen den Batch mit dem Index 5 aus dem Testset (indices_test[5])
# Sie können hier jeden Index von 0 bis len(X_test_scaled)-1 wählen
sample_index = 5
# sample_index = random.randint(0, len(X_test_scaled) - 1)
sample_batch_scaled = X_test_scaled[sample_index]
sample_target_scaled = y_test_scaled[sample_index]

# Hol den wahren Endwert und skaliere ihn zurück, um ihn später vergleichen zu können
true_final_value = scaler_y.inverse_transform(sample_target_scaled.reshape(1, -1))[0, 0]

print(f"Demonstration für einen einzelnen Batch aus dem Testset.")
print(
    f"Der wahre Endwert (Qualitätsvariable) für diesen Batch ist: {true_final_value:.2f}"
)
print("-" * 50)


# 2. DER VORHERSAGE-PROZESS

# Wir machen Vorhersagen zu verschiedenen Zeitpunkten
evaluation_points = np.linspace(0.1, 1.0, 10)  # 10%, 20%, ..., 100%
timesteps = sample_batch_scaled.shape[0]  # 140
predictions_over_time = []

for progress in evaluation_points:
    # Nimm nur die bisher verfügbaren Daten des Batches
    current_timesteps = int(timesteps * progress)
    partial_sequence = sample_batch_scaled[:current_timesteps, :]

    # Das Modell erwartet immer einen Input der Länge 140.
    # Wir müssen die unvollständige Sequenz mit Nullen auffüllen (Padding).
    # Wir fügen die Nullen am Anfang hinzu ('pre'), da die neuesten Daten am wichtigsten sind.
    padded_sequence = pad_sequences(
        [partial_sequence],  # Muss in einer Liste sein
        maxlen=timesteps,
        dtype="float32",
        padding="pre",
    )

    # Mache die Vorhersage. Das `[0, 0]` am Ende extrahiert den einzelnen Zahlenwert.
    prediction_scaled = model.predict(padded_sequence)[0, 0]

    # Skaliere die Vorhersage zurück in die ursprüngliche, interpretierbare Einheit
    prediction_real_value = scaler_y.inverse_transform([[prediction_scaled]])[0, 0]

    predictions_over_time.append(prediction_real_value)

    print(
        f"Nach {progress*100:.0f}% des Batches: Vorhersage für Endwert = {prediction_real_value:.2f}"
    )


# 3. VISUALISIERUNG DES ERGEBNISSES

# Erstelle ein DataFrame für den Plot
prediction_df = pd.DataFrame(
    {
        "Batch-Fortschritt (%)": evaluation_points * 100,
        "Vorhergesagter Endwert": predictions_over_time,
    }
)

# Erstelle die Figur
fig = go.Figure()

# Füge die Linie für die Vorhersagen hinzu
fig.add_trace(
    go.Scatter(
        x=prediction_df["Batch-Fortschritt (%)"],
        y=prediction_df["Vorhergesagter Endwert"],
        mode="lines+markers",
        name="Soft-Sensor-Vorhersage",
    )
)

# Füge eine horizontale Linie für den wahren Endwert hinzu
fig.add_hline(
    y=true_final_value,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Wahrer Endwert: {true_final_value}",
    annotation_position="bottom right",
)

fig.update_layout(
    title="Live-Vorhersage des Soft Sensors für einen einzelnen Batch",
    xaxis_title="Verfügbare Daten vom Batch (%)",
    yaxis_title="Vorhergesagtes Gesamtvolumen V_total_out",
)

fig.show()

c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning:

`build()` was called on layer 'spatial_attention_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.

c:\Users\sinan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning:

`build()` was called on layer 'temporal_attention_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.



Demonstration für einen einzelnen Batch aus dem Testset.
Der wahre Endwert (Qualitätsvariable) für diesen Batch ist: 1037.05
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
Nach 10% des Batches: Vorhersage für Endwert = 839.08
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Nach 20% des Batches: Vorhersage für Endwert = 818.71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Nach 30% des Batches: Vorhersage für Endwert = 819.65
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Nach 40% des Batches: Vorhersage für Endwert = 833.56
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Nach 50% des Batches: Vorhersage für Endwert = 848.24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Nach 60% des Batches: Vorhersage für Endwert = 871.21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Nach 70% des Batches: Vorhersage für Endwert = 899.99
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Nach 80% des Batches: Vorhersage für Endwert = 941.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Nach 90% des Batches: Vorhersage für Endwert = 98